import dependencies

In [3]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, SparseVectorParams, Document, Prefetch, FusionQuery
from qdrant_client import models

import pandas as pd
import openai
import fastembed

/Users/camilo/retrieval_rec_amazon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
qdrant_client = QdrantClient(url="http://localhost:6333")

Qdrant Client collection for Hybrid Search

In [29]:
from turtle import distance


qdrant_client.create_collection(
    collection_name = "Amazon-items-collection-02-hybrid-search",
    vectors_config= {
        "text-embedding-3-small": VectorParams(size=1536, distance= Distance.COSINE)
    },
    sparse_vectors_config= {
        "bm25": SparseVectorParams(modifier=models.Modifier.IDF)
    }
)

True

In [30]:
qdrant_client.create_payload_index(
    collection_name= "Amazon-items-collection-02-hybrid-search",
    field_name = "parent_asin",
    field_schema = PayloadSchemaType.KEYWORD
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

##Embedding Functions

In [5]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding

In [19]:
def get_embedding_batch(text_list, model="text-embedding-3-small", batch_size=100):

    if len(text_list) <= batch_size:
        response = openai.embeddings.create(input= text_list, model=model)
        return [embedding.embedding for embedding in response.data]
    
    all_embeddings = []
    counter = 1
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i+batch_size]
        response = openai.embeddings.create(input= batch, model = model)
        all_embeddings.extend([embedding.embedding for embedding in response.data])
        print(f"Processed {counter + batch_size} of {len(text_list)}")
        counter +=1
    return all_embeddings


Process and Embed Amazon Items Data

In [5]:
df_items = pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)

In [6]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,[],4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN
2,All Electronics,USB C Docking Station Dual Monitor for MacBook...,3.9,1193,[【18-in-1Docking Station】With USB C Docking St...,[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],ZMUIPNG,"[Electronics, Computers & Accessories, Laptop ...","{'Product Dimensions': '3.94""L x 1.18""W x 3.94...",B09SFN9NRX,NaN,NaN,NaN
3,Camera & Photo,[2023 Upgraded] Telescopes for Adults Astronom...,4.1,219,[🎁【2023 All New Experience】The newly upgraded ...,[],169.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Good picture quality', 'url': 'htt...",HUTACT,"[Electronics, Camera & Photo, Binoculars & Sco...","{'Product Dimensions': '32.5""D x 5.5""W x 9.7""H...",B09TP3SZ7C,NaN,NaN,NaN
4,AMAZON FASHION,"Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",4.5,222,"[Leather,Mesh, Imported, Multi-pockets and Lar...",[],24.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],KPIQIU,"[Electronics, Computers & Accessories, Laptop ...",{'Product Dimensions': '16 x 2 x 12 inches; 1....,B0B5H7T7XZ,NaN,NaN,NaN


In [9]:
df_items.shape

(1000, 16)

In [10]:
def preprocess_description(row):
    return f"{row['title']} {' '.join(row['features'])}"

In [11]:
def extract_first_large_image(row):
    return row["images"][0].get("large", "")

In [12]:
df_items["description"] = df_items.apply(preprocess_description, axis=1)
df_items["image"] = df_items.apply(extract_first_large_image, axis=1)

In [13]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,image
0,Industrial & Scientific,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",4.4,119,[【Fast Charging Cord】These USB C cables provid...,"RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB T...",NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Type-C Charger Cable ', 'url': 'ht...",RAVODOI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'RAVODOI', 'Connector Type': 'USB Ty...",B09R4Y2HKY,NaN,NaN,NaN,https://m.media-amazon.com/images/I/51G07yWoOB...
1,All Electronics,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.5,352,[🔹(Light & compact) Easy to carry and light we...,"SNESH-2 Pack USB-C Female to USB Male Adapter,...",4.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'USB Male & Female Adapter', 'url':...",SNESH,"[Electronics, Computers & Accessories, Compute...",{'Package Dimensions': '3.54 x 2.4 x 0.35 inch...,B09JV5FM2S,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41bOA5-ogW...
2,All Electronics,USB C Docking Station Dual Monitor for MacBook...,3.9,1193,[【18-in-1Docking Station】With USB C Docking St...,USB C Docking Station Dual Monitor for MacBook...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],ZMUIPNG,"[Electronics, Computers & Accessories, Laptop ...","{'Product Dimensions': '3.94""L x 1.18""W x 3.94...",B09SFN9NRX,NaN,NaN,NaN,https://m.media-amazon.com/images/I/416IzmVKiC...
3,Camera & Photo,[2023 Upgraded] Telescopes for Adults Astronom...,4.1,219,[🎁【2023 All New Experience】The newly upgraded ...,[2023 Upgraded] Telescopes for Adults Astronom...,169.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Good picture quality', 'url': 'htt...",HUTACT,"[Electronics, Camera & Photo, Binoculars & Sco...","{'Product Dimensions': '32.5""D x 5.5""W x 9.7""H...",B09TP3SZ7C,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41wO4J3TT0...
4,AMAZON FASHION,"Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",4.5,222,"[Leather,Mesh, Imported, Multi-pockets and Lar...","Laptop Bag 15.6 Inch, Laptop Briefcase Messeng...",24.95,[{'thumb': 'https://m.media-amazon.com/images/...,[],KPIQIU,"[Electronics, Computers & Accessories, Laptop ...",{'Product Dimensions': '16 x 2 x 12 inches; 1....,B0B5H7T7XZ,NaN,NaN,NaN,https://m.media-amazon.com/images/I/41mwlYqT5p...


In [14]:
data_to_embed = df_items[["description", "image", "rating_number", "price", "average_rating", "parent_asin"]].to_dict(orient="records")

In [15]:
data_to_embed

[{'description': "RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB Type C Fast Charging Cord - Nylon Braided USB C Charger Cable for Galaxy A20/A50/S10/S9/S8+/S8, iPad Pro 2018, Sony XZ, HTC 10, OnePlus 5T, Huawei P9 etc. 【Fast Charging Cord】These USB C cables provide up to a 3A charging current to greatly shorten the charging time, meets QC2.0 /3.0 fast charging protocol,Incredibly charge your phone from 0 to 80% in 50 minute. 480Mbps (40-60M/s) ultra fast data transmission, which leads to a faster data sync.(Note:Cables support fast charging,but require a USB-A QC3.0/QC2.0/AFC charger) 【Universal Compatibility】The USB C Charger Cable is compatible with Samsung Galaxy S20 / S10 / S9 / S8+ / S8 / A02s / A03s,A12 A20 A21 A22 A23 A31 A32 A33 A41 A42 A50 A52 A52s 5G A71 A72 A73,M11 M21 M31 M51,M12 M22 M32 M52,iPad Pro 2018 / 2020,Sony Xperia XZ/X Compact/L1 / XZs / XA1 / X Premium, HTC 10 LG G5 G6,OnePlus 5T / 6T, Lumia 950 / 950XL,Huawei P9 P9 Plus P10 P10 Plus Honor Mate 9 Mate 9 pro Mate 1

In [16]:
text_to_embed = [data['description'] for data in data_to_embed]

In [17]:
text_to_embed

["RAVODOI USB C Cable, [2Pack/3.3ft+6.6ft] USB Type C Fast Charging Cord - Nylon Braided USB C Charger Cable for Galaxy A20/A50/S10/S9/S8+/S8, iPad Pro 2018, Sony XZ, HTC 10, OnePlus 5T, Huawei P9 etc. 【Fast Charging Cord】These USB C cables provide up to a 3A charging current to greatly shorten the charging time, meets QC2.0 /3.0 fast charging protocol,Incredibly charge your phone from 0 to 80% in 50 minute. 480Mbps (40-60M/s) ultra fast data transmission, which leads to a faster data sync.(Note:Cables support fast charging,but require a USB-A QC3.0/QC2.0/AFC charger) 【Universal Compatibility】The USB C Charger Cable is compatible with Samsung Galaxy S20 / S10 / S9 / S8+ / S8 / A02s / A03s,A12 A20 A21 A22 A23 A31 A32 A33 A41 A42 A50 A52 A52s 5G A71 A72 A73,M11 M21 M31 M51,M12 M22 M32 M52,iPad Pro 2018 / 2020,Sony Xperia XZ/X Compact/L1 / XZs / XA1 / X Premium, HTC 10 LG G5 G6,OnePlus 5T / 6T, Lumia 950 / 950XL,Huawei P9 P9 Plus P10 P10 Plus Honor Mate 9 Mate 9 pro Mate 10 pro Mate 10 li

In [20]:
embeddings = get_embedding_batch(text_to_embed)

Processed 101 of 1000
Processed 102 of 1000
Processed 103 of 1000
Processed 104 of 1000
Processed 105 of 1000
Processed 106 of 1000
Processed 107 of 1000
Processed 108 of 1000
Processed 109 of 1000
Processed 110 of 1000


In [22]:
embeddings[0]

[0.013903025537729263,
 -0.03039454109966755,
 -0.014772596769034863,
 0.010025345720350742,
 -0.04509636014699936,
 -0.02723981812596321,
 -0.048898205161094666,
 0.023458195850253105,
 0.06163843348622322,
 -0.03075854852795601,
 -0.014408590272068977,
 -0.002176455454900861,
 -0.06147665157914162,
 -0.02616802230477333,
 0.0340346060693264,
 0.02182016707956791,
 -0.05160802975296974,
 -0.026552250608801842,
 -0.024914221838116646,
 0.007497522514313459,
 0.008569319732487202,
 0.020151803269982338,
 0.021981947124004364,
 0.019454125314950943,
 -0.0268353670835495,
 -0.03225501999258995,
 -0.018513774499297142,
 0.028554286807775497,
 -0.012993009760975838,
 -0.009332722052931786,
 -0.04582437127828598,
 -0.02218417264521122,
 0.004686583764851093,
 -0.020525921136140823,
 -0.04695683717727661,
 0.05670412257313728,
 -0.01109208632260561,
 0.005535931792110205,
 0.03720955178141594,
 -0.009064773097634315,
 0.01648140512406826,
 0.005682545714080334,
 0.04966666176915169,
 -0.02910

In [23]:
len(embeddings)

1000

In [24]:
pointstructs = []
i = 1
for embedding, data in zip(embeddings, data_to_embed):
    pointstructs.append(
        PointStruct(
            id = i, 
            vector = {
                "text-embedding-3-small": embedding,
                "bm25": Document(
                    text=data["description"],
                    model="qdrant/bm25"
                )
            },
            payload=data
        )

    )
    i += 1

In [25]:
pointstructs[0].vector

{'text-embedding-3-small': [0.013903025537729263,
  -0.03039454109966755,
  -0.014772596769034863,
  0.010025345720350742,
  -0.04509636014699936,
  -0.02723981812596321,
  -0.048898205161094666,
  0.023458195850253105,
  0.06163843348622322,
  -0.03075854852795601,
  -0.014408590272068977,
  -0.002176455454900861,
  -0.06147665157914162,
  -0.02616802230477333,
  0.0340346060693264,
  0.02182016707956791,
  -0.05160802975296974,
  -0.026552250608801842,
  -0.024914221838116646,
  0.007497522514313459,
  0.008569319732487202,
  0.020151803269982338,
  0.021981947124004364,
  0.019454125314950943,
  -0.0268353670835495,
  -0.03225501999258995,
  -0.018513774499297142,
  0.028554286807775497,
  -0.012993009760975838,
  -0.009332722052931786,
  -0.04582437127828598,
  -0.02218417264521122,
  0.004686583764851093,
  -0.020525921136140823,
  -0.04695683717727661,
  0.05670412257313728,
  -0.01109208632260561,
  0.005535931792110205,
  0.03720955178141594,
  -0.009064773097634315,
  0.016481

In [31]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-02-hybrid-search",
    points=pointstructs[:500],
    wait=True,
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [32]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-02-hybrid-search",
    points=pointstructs[500:],
    wait=True,
)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

<h3>Hybrid Retrieval</h4>

In [39]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )


    return response.data[0].embedding

In [40]:
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-02-hybrid-search",
        prefetch = [
            Prefetch(
                query= query_embedding,
                using = "text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(
                    text=query,
                    model="qdrant/bm25"
                ),
                using = "bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


In [41]:
results = retrieve_data("Can I get some tablets", qdrant_client, k=20)

In [42]:
results

{'retrieved_context_ids': ['B0B8NVNQKX',
  'B0C1NP5KYD',
  'B09LH466KZ',
  'B0BN7WWH63',
  'B0B6ZZH83Y',
  'B0BL2CZSHT',
  'B08CGM6Y1J',
  'B0C78B1BTB',
  'B0BR33XH8D',
  'B0B4WCNWF3',
  'B0BFGXRMJN',
  'B0C3LNX75K',
  'B0BVZ47N6N',
  'B0C3XYD574',
  'B0BP1YXF3R',
  'B0C772GZRF',
  'B0B965NYT9',
  'B09XQGN52P',
  'B0BDZNFFLT',
  'B0BS15TRJ3'],
 'retrieved_context': ['COOPERS 7 inch Kids Tablet Android 11 Tablet for Kids, 2GB RAM + 32GB ROM Toddler Tablet PC for Children, IPS Touch Screen, Dual Camera, Dual Speaker, WiFi Computer Tablet, Light Blue ✿【Good Kids tablet】This 7 inch tablet for kids with silm body and lightweight, it is easy to hold by children. Also the special design can protect the tablet well when dropping. ✿【Parental Control】Toddlers Tablet with parent mode can add or block apps. Set screen time limits. This tablet come with iwawa app. kids can get access to fun and educational games and videos. ✿【Powerful Tablets】Equipmented with quad core CPU, Android 11.0 System, 32G